### Import library

In [63]:
from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType

import os
import zipfile
import json
import pandas as pd

In [6]:
esg_path = "D:/jyp/kpmg_co/1. preprocessing/1-2. ESG report/"

In [43]:
input_pdf_list = os.listdir(esg_path+'esg_report/')

In [44]:
input_pdf_list

['GSretail_2023.pdf',
 'hyundaehomeshopping_2023.pdf',
 'LGlife_2023.pdf',
 'lottehi_2023.pdf',
 'sinsegye_2023.pdf']

In [47]:
zip_file_name_list = []

for i in range(len(input_pdf_list)):
    zip_file_name_list.append(input_pdf_list[i].replace('.pdf', ''))

In [48]:
zip_file_name_list

['GSretail_2023',
 'hyundaehomeshopping_2023',
 'LGlife_2023',
 'lottehi_2023',
 'sinsegye_2023']

### PDF에서 table, text, figure 추출하기

In [49]:

for i in range(len(input_pdf_list)):
    
    # 파일 불러오기 
    zip_file = esg_path + "ExtractTextInfoFromPDF_{}.zip".format(zip_file_name_list[i])

    if os.path.isfile(zip_file):
        os.remove(zip_file)

    input_pdf = esg_path +"esg_report/{}".format(input_pdf_list[i])
    
    #Initial setup, create credentials instance.
    credentials = Credentials.service_account_credentials_builder()\
        .from_file(esg_path+"adobe-dc-pdf-services-sdk-extract-python-samples/pdfservices-api-credentials.json") \
        .build()

    #Create an ExecutionContext using credentials and create a new operation instance.
    execution_context = ExecutionContext.create(credentials)
    
    extract_pdf_operation = ExtractPDFOperation.create_new()

    #Set operation input from a source file.
    source = FileRef.create_from_local_file(input_pdf)
    extract_pdf_operation.set_input(source)

    #Build ExtractPDF options and set them into the operation
    extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
        .with_element_to_extract(ExtractElementType.TEXT) \
        .build()
    extract_pdf_operation.set_options(extract_pdf_options)
    
    #Execute the operation.
    result: FileRef = extract_pdf_operation.execute(execution_context)

    #Save the result to the specified location.
    result.save_as(zip_file)

### esg 관련 문장 추출 및 csv로 저장

In [96]:
for i in range(len(input_pdf_list)):
    output_zip = esg_path +"ExtractTextInfoFromPDF_{}.zip".format(zip_file_name_list[i])
    archive = zipfile.ZipFile(output_zip, 'r')
    jsonentry = archive.open('structuredData.json')
    jsondata = jsonentry.read()
    data = json.loads(jsondata)
    empty_df = []
    empty_df_2 = []

    for j in range(len(data["elements"])):
        if 'Text' in data["elements"][j]:
            if len(data["elements"][j]['Text']) > 50:
                empty_df.append(data["elements"][j]['Text'])
            else:
                pass
        else:
            pass

    for k in range(len(empty_df)):
        if '' in empty_df[k]:
            pass
        else : 
            empty_df_2.append(empty_df[k])
            
    str_df = pd.DataFrame(empty_df_2, columns=['text'])
    
    # output 폴더 없으면 만들기 
    if not os.path.isdir(esg_path + 'output/'):
        os.mkdir(esg_path + 'output/')
    str_df.to_csv(esg_path + 'output/'+"ESG_text_{}.csv".format(zip_file_name_list[i]), index=False, encoding='utf-8-sig')

In [90]:
str_df = pd.DataFrame(empty_df_2, columns=['text'])

In [91]:
str_df

,text
0,이사회는 최고의사결정기구로 회사 경영 기본방침 및 중요사항을 의결하 고 있으며 사내...
1,"칙으로 합니다. 필요시 임시이사회1)를 개최하여 운영하고, 이사회 결의는 정 함이 ..."
2,"사내이사는 상법 제382조에 따라 주주총회에서 선임하며, 사내이사는 이사 회에서 경..."
3,"이사 선임 시 금융, 법률, 회계 등 관련분야 전문지식과 실무경험이 풍부한 자 로 ..."
4,"경영진과 지배주주로부터 독립적 이사회 기능을 수행할 수 있도록, 정관 제 26조에 ..."
...,...
79,이 실무에서도 공정거래 준수를 생활화할 수 있도록 2021년 반기 1회 공정거 래 ...
80,롯데하이마트는 파트너사와 환경 및 안전 관련 파트너십을 구축하여 환경영 향 최소화를...
81,유해화학물질의 안전한 관리와 유출 방지를 위해 파트너사에 대한 지속적인 교육 및 점...
82,환경 유해물질을 배출할 가능성이 있는 파트너사를 대상으로 연 1회 안전환 경 교육 ...
